# Investigating the State Anomaly Detection

In [1]:
import os
import sys
import torch
import numpy as np
import pandas as pd
from time import time
from scipy import stats
from tabulate import tabulate

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, classification_report, accuracy_score

/Users/jankreischer/opt/anaconda3/envs/FedRL-for-IT-Sec/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
from enum import Enum

class Behavior(Enum):
    NORMAL = "normal"
    ROOTKIT_BDVL = "bdvl"
    ROOTKIT_BEURK = "beurk"
    CNC_BACKDOOR_JAKORITAR = "backdoor_jakoritar"
    CNC_THETICK = "the_tick"
    CNC_OPT1 = "data_leak_1"
    CNC_OPT2 = "data_leak_2"
    RANSOMWARE_POC = "ransomware_poc"


In [2]:
dataset = pd.read_csv('dataset-01.csv')        

In [71]:
print(f"Full length of dataset: {len(dataset)}")

Full length of dataset: 59004


In [3]:
# Dropping time status features
time_status_columns = ['time', 'timestamp', 'seconds']
try:
    dataset.drop(time_status_columns, inplace=True, axis=1)
except:
    print(f"All time status features {(time_status_columns)} are removed from the dataset")

In [4]:
from fast_ml.feature_selection import get_constant_features

# Removing constant features
constant_features = set(get_constant_features(dataset, threshold=0.99, dropna=False)['Var'])
try:
    dataset.drop(constant_features, inplace=True, axis=1)
except:
    print(f"All constant features {(constant_features)} are removed from the dataset")

In [55]:
from sklearn.preprocessing import MinMaxScaler
# Scaling
fit_normal_behavior_only = True
standard_scaling = False
if standard_scaling:
    scaler = StandardScaler()
else:
    scaler = MinMaxScaler()
    
print(f"Using {scaler}")

if fit_normal_behavior_only: 
    scaler.fit(dataset[dataset['behavior'] == "Behavior.NORMAL"].values[:,:-1])
else: 
    scaler.fit(dataset.values[:,:-1])

scaled_dataset = pd.DataFrame(scaler.transform(dataset.values[:,:-1]), columns=dataset.columns.drop("behavior"), index=dataset.index)
scaled_dataset["behavior"] = dataset["behavior"]

Using MinMaxScaler()


In [146]:
X_normal = scaled_dataset.loc[scaled_dataset['behavior'] == "Behavior.NORMAL"].drop(["behavior"],  axis=1).to_numpy()
X_normal.shape

(14702, 85)

In [136]:
import torch.nn as nn

class RMSELoss(nn.Module):
    def __init__(self, eps=1e-6):
        super().__init__()
        self.mse = torch.nn.MSELoss()
        self.eps = eps
        
    def forward(self, yhat, y):
        loss = torch.sqrt(self.mse(yhat,y) + self.eps)
        return loss

class AutoEncoder(torch.nn.Module):
    

    def __init__(self, n_features):
        super().__init__()

        self.model = nn.Sequential(
        nn.Linear(n_features, 64),
        nn.BatchNorm1d(64),
        nn.GELU(),
        nn.Linear(64, 20),
        nn.GELU(),
        #nn.Linear(32, 16),
        #nn.GELU(),
        #nn.Linear(16, 32),
        #nn.GELU(),
        nn.Linear(20, 64),
        nn.BatchNorm1d(64),
        nn.GELU(),
        nn.Linear(64, n_features),
        nn.GELU()
    )
        print("Usring 20")
        self.threshold = None
        self.loss_mean = None
        self.loss_standard_deviation = None
        self.prediction_loss_function = torch.nn.MSELoss(reduction="sum")
        
        
    def forward(self, X):
        return self.model(X)
    
    
    def pretrain(self, X_normal, optimizer=torch.optim.SGD, loss_function=torch.nn.MSELoss(reduction='mean'), num_epochs: int = 15, batch_size=64, verbose=False):
        threshold = int(0.5*len(X_normal))
        X_train = X_normal[:threshold]
        X_valid = X_normal[threshold:]
        
        training_dataset = torch.utils.data.TensorDataset(
            torch.from_numpy(X_train).type(torch.float),
        )
        training_data_loader = torch.utils.data.DataLoader(training_dataset, batch_size=batch_size, shuffle=True, drop_last=True)

        epoch_losses = []
        #for e in tqdm(range(num_epochs), unit="epoch", leave=False):
        for e in range(num_epochs):
            self.train()
            current_losses = []
            for batch_index, (inputs,) in enumerate(training_data_loader):
                optimizer.zero_grad()
                outputs = self.forward(inputs)
                loss = loss_function(inputs, outputs)
                loss.backward()
                optimizer.step()
                current_losses.append(loss.item())
            
            epoch_losses.append(np.average(current_losses))
            if verbose:
                print(f'Training Loss in epoch {e + 1}: {epoch_losses[e]}')
            
        self.analyze_loss(X_valid)

    '''
    This function uses normal data samles 
    after training the autoencoder to determine
    values that can be considered normal
    for the reconstruction loss based on normal samples
    '''
    def analyze_loss(self, X_valid):
        validation_dataset = torch.utils.data.TensorDataset(
            torch.from_numpy(X_valid).type(torch.float),

        )
        validation_data_loader = torch.utils.data.DataLoader(validation_dataset, batch_size=1, shuffle=True, drop_last=True)
        
        losses = []
        
        self.eval() 
        with torch.no_grad():
            for batch_index, (inputs,) in enumerate(validation_data_loader):
                outputs = self.forward(inputs)
                loss = self.prediction_loss_function(inputs, outputs)
                losses.append(loss.item())
        
        losses = np.array(losses)
        print(f"len(losses): {len(losses)}")
        self.loss_mean = losses.mean()
        self.loss_standard_deviation = losses.std()
        print(f"loss_mean: {self.loss_mean}")
        print(f"loss_standard_deviation: {self.loss_standard_deviation}")

        
    def predict(self, x, n_std=3):
        test_data = torch.utils.data.TensorDataset(
            torch.from_numpy(x).type(torch.float32)
        )
        test_data_loader = torch.utils.data.DataLoader(test_data, batch_size=1, shuffle=False)

        all_predictions = torch.tensor([])  # .cuda()

        self.eval()
        with torch.no_grad():
            for idx, (batch_x,) in enumerate(test_data_loader):
                model_predictions = self.forward(batch_x)
                model_predictions = self.prediction_loss_function(model_predictions, batch_x).unsqueeze(0)  # unsqueeze as batch_size set to 1
                all_predictions = torch.cat((all_predictions, model_predictions))

        threshold = self.loss_mean + n_std * self.loss_standard_deviation
        all_predictions = (all_predictions > threshold).type(torch.long)
        return all_predictions.flatten()
    
    
    def predict_deviation(self, x):
        test_data = torch.utils.data.TensorDataset(
            torch.from_numpy(x).type(torch.float32)
        )
        test_data_loader = torch.utils.data.DataLoader(test_data, batch_size=1, shuffle=False)

        prediction_errors = torch.tensor([])
        
        self.eval()
        with torch.no_grad():
            for batch_index, (inputs,) in enumerate(test_data_loader):
                prediction = self.forward(inputs)
                prediction_error = self.prediction_loss_function(inputs, prediction).unsqueeze(0)  # unsqueeze as batch_size set to 1
                prediction_errors = torch.cat((prediction_errors, prediction_error))

        return prediction_errors
    
    
    def score(self):
        n_std, accuracy = self.accuracy_score(None, None)
        if self.verbose:
            print(f"Highest validation accuracy achieved {accuracy:.2f} with n_std={n_std}")
            self.evaluate(n_std)
        return accuracy
    
    
    def accuracy_score(self, X, y):
        #if not self.threshold:
        #loss_mean, loss_standard_deviation = self.analyze_loss(X)
        #n_stds = np.arange(0.1, 3, 0.1)
        if self.loss_mean == None or self.loss_standard_deviation == None:
              #print("accuracy_score_optimized > accurcy_loss()")
              self.analyze_loss()
    
        best_accuracy = 0
        best_n_std = 0
        #accuracies = []
        y_dev = self.predict_deviation((self.X_test).astype(np.float32))
        for n_std in self.n_stds:
            y_true = self.y_test
            threshold = self.loss_mean + n_std * self.loss_standard_deviation
            y_pred = (y_dev > threshold).type(torch.long).detach().cpu().numpy()
            
            accuracy = accuracy_score(y_true, y_pred)
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_n_std = n_std
            #if self.verbose:
            #    print(f"n_std {n_std:.2f} -> accuracy: {accuracy}")

        return best_n_std, best_accuracy
    
    
    def evaluate(self, evaluation_data, n_std=3, tablefmt='pipe'):
        results = []
        labels= [0,1]
        pos_label = 1
        
        y_true_total = np.empty([0])
        y_pred_total = np.empty([0])
        for behavior in Behavior:
            X_behavior = evaluation_data.loc[evaluation_data['behavior'] == f"Behavior.{behavior.name}"].drop(["behavior"],  axis=1).to_numpy()
            print(len(X_behavior))
            y_true = np.array([0 if behavior == Behavior.NORMAL else 1] * len(X_behavior)).astype(int)
            y_true_total = np.concatenate((y_true_total, y_true))
            
            print(f"Using n_std: {n_std} as prediction threshold")
            y_pred = self.predict(X_behavior.astype(np.float32), n_std=n_std)
            y_pred_total = np.concatenate((y_pred_total, y_pred))

            accuracy = accuracy_score(y_true, y_pred)

            n_samples = len(y_true)
            results.append([behavior.name.replace("_", "\_"), f'{(100 * accuracy):.2f}\%', '\\notCalculated', '\\notCalculated', '\\notCalculated', str(n_samples)])

        accuracy = accuracy_score(y_true_total, y_pred_total)
        precision = precision_score(y_true_total, y_pred_total, average='binary', labels=labels, pos_label=pos_label, zero_division=1)
        recall = recall_score(y_true_total, y_pred_total, average='binary', labels=labels, pos_label=pos_label, zero_division=1)
        f1 = f1_score(y_true_total, y_pred_total, average='binary', labels=labels, pos_label=pos_label, zero_division=1)
        n_samples = len(y_true_total)
        results.append(["GLOBAL", f'{(100 * accuracy):.2f}\%', f'{(100 * precision):.2f}\%', f'{(100 * recall):.2f}\%', f'{(100 * f1):.2f}\%', n_samples])
        print("-----------")
        print(tabulate(results, headers=["Behavior", "Accuracy", "Precision", "Recall", "F1-Score", "\#Samples"], tablefmt=tablefmt)) 

In [137]:
N_FEATURES = X_normal.shape[1]
autoencoder = AutoEncoder(N_FEATURES)

Usring 20


In [138]:
autoencoder.pretrain(X_normal, optimizer=torch.optim.Adam(autoencoder.parameters(), lr=1e-4,  weight_decay=0.01), loss_function=RMSELoss(), num_epochs=100, batch_size=64, verbose=False)

len(losses): 7351
loss_mean: 0.28526486205997525
loss_standard_deviation: 0.2960638965343553


In [145]:
# With batch norm and GELU and 64 x 20 nodes
evaluation_data = scaled_dataset
autoencoder.evaluate(evaluation_data, n_std=0.7, tablefmt='latex_raw')

14702
Using n_std: 0.7 as prediction threshold
5698
Using n_std: 0.7 as prediction threshold
7358
Using n_std: 0.7 as prediction threshold
4312
Using n_std: 0.7 as prediction threshold
7704
Using n_std: 0.7 as prediction threshold
5687
Using n_std: 0.7 as prediction threshold
4162
Using n_std: 0.7 as prediction threshold
9381
Using n_std: 0.7 as prediction threshold
-----------
\begin{tabular}{lllllr}
\hline
 Behavior                 & Accuracy   & Precision      & Recall         & F1-Score       &   \#Samples \\
\hline
 NORMAL                   & 93.65\%    & \notCalculated & \notCalculated & \notCalculated &       14702 \\
 ROOTKIT\_BDVL            & 100.00\%   & \notCalculated & \notCalculated & \notCalculated &        5698 \\
 ROOTKIT\_BEURK           & 100.00\%   & \notCalculated & \notCalculated & \notCalculated &        7358 \\
 CNC\_BACKDOOR\_JAKORITAR & 100.00\%   & \notCalculated & \notCalculated & \notCalculated &        4312 \\
 CNC\_THETICK             & 100.00\%   & \notC

In [144]:
# With batch norm and GELU and 64 x 16 nodes
evaluation_data = scaled_dataset
autoencoder.evaluate(evaluation_data, n_std=0.75, tablefmt='latex_raw')

14702
Using n_std: 0.75 as prediction threshold


KeyboardInterrupt: 

In [127]:
autoencoder.pretrain(X_normal, optimizer=torch.optim.Adam(autoencoder.parameters(), lr=1e-4,  weight_decay=0.01), loss_function=RMSELoss(), num_epochs=100, batch_size=64, verbose=False)

len(losses): 7351
loss_mean: 0.3091465074915113
loss_standard_deviation: 0.32274408025982587


In [128]:
# With batch norm and GELU and 64 x 32 nodes
evaluation_data = scaled_dataset
autoencoder.evaluate(evaluation_data, n_std=0.5, tablefmt='latex_raw')

14702
Using n_std: 0.5 as prediction threshold
5698
Using n_std: 0.5 as prediction threshold
7358
Using n_std: 0.5 as prediction threshold
4312
Using n_std: 0.5 as prediction threshold
7704
Using n_std: 0.5 as prediction threshold
5687
Using n_std: 0.5 as prediction threshold
4162
Using n_std: 0.5 as prediction threshold
9381
Using n_std: 0.5 as prediction threshold
-----------
\begin{tabular}{lllllr}
\hline
 Behavior                 & Accuracy   & Precision      & Recall         & F1-Score       &   \#Samples \\
\hline
 NORMAL                   & 90.24\%    & \notCalculated & \notCalculated & \notCalculated &       14702 \\
 ROOTKIT\_BDVL            & 100.00\%   & \notCalculated & \notCalculated & \notCalculated &        5698 \\
 ROOTKIT\_BEURK           & 100.00\%   & \notCalculated & \notCalculated & \notCalculated &        7358 \\
 CNC\_BACKDOOR\_JAKORITAR & 100.00\%   & \notCalculated & \notCalculated & \notCalculated &        4312 \\
 CNC\_THETICK             & 100.00\%   & \notC

In [124]:
# With batch norm and GELU
evaluation_data = scaled_dataset
autoencoder.evaluate(evaluation_data, n_std=0.5, tablefmt='latex_raw')

14702
Using n_std: 0.5 as prediction threshold
5698
Using n_std: 0.5 as prediction threshold
7358
Using n_std: 0.5 as prediction threshold
4312
Using n_std: 0.5 as prediction threshold
7704
Using n_std: 0.5 as prediction threshold
5687
Using n_std: 0.5 as prediction threshold
4162
Using n_std: 0.5 as prediction threshold
9381
Using n_std: 0.5 as prediction threshold
-----------
\begin{tabular}{lllllr}
\hline
 Behavior                 & Accuracy   & Precision      & Recall         & F1-Score       &   \#Samples \\
\hline
 NORMAL                   & 86.71\%    & \notCalculated & \notCalculated & \notCalculated &       14702 \\
 ROOTKIT\_BDVL            & 54.35\%    & \notCalculated & \notCalculated & \notCalculated &        5698 \\
 ROOTKIT\_BEURK           & 100.00\%   & \notCalculated & \notCalculated & \notCalculated &        7358 \\
 CNC\_BACKDOOR\_JAKORITAR & 100.00\%   & \notCalculated & \notCalculated & \notCalculated &        4312 \\
 CNC\_THETICK             & 100.00\%   & \notC

In [120]:
# With batch norm
evaluation_data = scaled_dataset
autoencoder.evaluate(evaluation_data, n_std=0.5, tablefmt='latex_raw')

14702
Using n_std: 0.5 as prediction threshold
5698
Using n_std: 0.5 as prediction threshold
7358
Using n_std: 0.5 as prediction threshold
4312
Using n_std: 0.5 as prediction threshold
7704
Using n_std: 0.5 as prediction threshold
5687
Using n_std: 0.5 as prediction threshold
4162
Using n_std: 0.5 as prediction threshold
9381
Using n_std: 0.5 as prediction threshold
-----------
\begin{tabular}{lllllr}
\hline
 Behavior                 & Accuracy   & Precision      & Recall         & F1-Score       &   \#Samples \\
\hline
 NORMAL                   & 84.93\%    & \notCalculated & \notCalculated & \notCalculated &       14702 \\
 ROOTKIT\_BDVL            & 91.72\%    & \notCalculated & \notCalculated & \notCalculated &        5698 \\
 ROOTKIT\_BEURK           & 100.00\%   & \notCalculated & \notCalculated & \notCalculated &        7358 \\
 CNC\_BACKDOOR\_JAKORITAR & 100.00\%   & \notCalculated & \notCalculated & \notCalculated &        4312 \\
 CNC\_THETICK             & 100.00\%   & \notC

In [113]:
# With batch norm
evaluation_data = scaled_dataset
autoencoder.evaluate(evaluation_data, n_std=0.5, tablefmt='latex_raw')

14702
Using n_std: 0.5 as prediction threshold
5698
Using n_std: 0.5 as prediction threshold
7358
Using n_std: 0.5 as prediction threshold
4312
Using n_std: 0.5 as prediction threshold
7704
Using n_std: 0.5 as prediction threshold
5687
Using n_std: 0.5 as prediction threshold
4162
Using n_std: 0.5 as prediction threshold
9381
Using n_std: 0.5 as prediction threshold
-----------
\begin{tabular}{lllllr}
\hline
 Behavior                 & Accuracy   & Precision      & Recall         & F1-Score       &   \#Samples \\
\hline
 NORMAL                   & 89.22\%    & \notCalculated & \notCalculated & \notCalculated &       14702 \\
 ROOTKIT\_BDVL            & 99.02\%    & \notCalculated & \notCalculated & \notCalculated &        5698 \\
 ROOTKIT\_BEURK           & 100.00\%   & \notCalculated & \notCalculated & \notCalculated &        7358 \\
 CNC\_BACKDOOR\_JAKORITAR & 100.00\%   & \notCalculated & \notCalculated & \notCalculated &        4312 \\
 CNC\_THETICK             & 100.00\%   & \notC

In [109]:
# No batch norm
evaluation_data = scaled_dataset
autoencoder.evaluate(evaluation_data, n_std=0.5, tablefmt='latex_raw')

14702
Using n_std: 0.5 as prediction threshold
5698
Using n_std: 0.5 as prediction threshold
7358
Using n_std: 0.5 as prediction threshold
4312
Using n_std: 0.5 as prediction threshold
7704
Using n_std: 0.5 as prediction threshold
5687
Using n_std: 0.5 as prediction threshold
4162
Using n_std: 0.5 as prediction threshold
9381
Using n_std: 0.5 as prediction threshold
-----------
\begin{tabular}{lllllr}
\hline
 Behavior                 & Accuracy   & Precision      & Recall         & F1-Score       &   \#Samples \\
\hline
 NORMAL                   & 84.32\%    & \notCalculated & \notCalculated & \notCalculated &       14702 \\
 ROOTKIT\_BDVL            & 25.94\%    & \notCalculated & \notCalculated & \notCalculated &        5698 \\
 ROOTKIT\_BEURK           & 100.00\%   & \notCalculated & \notCalculated & \notCalculated &        7358 \\
 CNC\_BACKDOOR\_JAKORITAR & 100.00\%   & \notCalculated & \notCalculated & \notCalculated &        4312 \\
 CNC\_THETICK             & 100.00\%   & \notC

In [94]:
# 8 neurons bottlekneck
evaluation_data = scaled_dataset
autoencoder.evaluate(evaluation_data, n_std=0.5, tablefmt='latex_raw')

14702
Using n_std: 0.5 as prediction threshold
5698
Using n_std: 0.5 as prediction threshold
7358
Using n_std: 0.5 as prediction threshold
4312
Using n_std: 0.5 as prediction threshold
7704
Using n_std: 0.5 as prediction threshold
5687
Using n_std: 0.5 as prediction threshold
4162
Using n_std: 0.5 as prediction threshold
9381
Using n_std: 0.5 as prediction threshold
-----------
\begin{tabular}{lllllr}
\hline
 Behavior                 & Accuracy   & Precision      & Recall         & F1-Score       &   \#Samples \\
\hline
 NORMAL                   & 84.39\%    & \notCalculated & \notCalculated & \notCalculated &       14702 \\
 ROOTKIT\_BDVL            & 26.54\%    & \notCalculated & \notCalculated & \notCalculated &        5698 \\
 ROOTKIT\_BEURK           & 100.00\%   & \notCalculated & \notCalculated & \notCalculated &        7358 \\
 CNC\_BACKDOOR\_JAKORITAR & 100.00\%   & \notCalculated & \notCalculated & \notCalculated &        4312 \\
 CNC\_THETICK             & 100.00\%   & \notC

In [101]:
# 16 neurons bottlekneck
evaluation_data = scaled_dataset
autoencoder.evaluate(evaluation_data, n_std=0.5, tablefmt='latex_raw')

14702
Using n_std: 0.5 as prediction threshold
5698
Using n_std: 0.5 as prediction threshold
7358
Using n_std: 0.5 as prediction threshold
4312
Using n_std: 0.5 as prediction threshold
7704
Using n_std: 0.5 as prediction threshold
5687
Using n_std: 0.5 as prediction threshold
4162
Using n_std: 0.5 as prediction threshold
9381
Using n_std: 0.5 as prediction threshold
-----------
\begin{tabular}{lllllr}
\hline
 Behavior                 & Accuracy   & Precision      & Recall         & F1-Score       &   \#Samples \\
\hline
 NORMAL                   & 84.34\%    & \notCalculated & \notCalculated & \notCalculated &       14702 \\
 ROOTKIT\_BDVL            & 26.29\%    & \notCalculated & \notCalculated & \notCalculated &        5698 \\
 ROOTKIT\_BEURK           & 100.00\%   & \notCalculated & \notCalculated & \notCalculated &        7358 \\
 CNC\_BACKDOOR\_JAKORITAR & 100.00\%   & \notCalculated & \notCalculated & \notCalculated &        4312 \\
 CNC\_THETICK             & 100.00\%   & \notC

In [85]:
autoencoder.pretrain(X_normal, optimizer=torch.optim.Adam(autoencoder.parameters(), lr=1e-4,  weight_decay=0.01), loss_function=RMSELoss(), num_epochs=100, batch_size=64, verbose=False)

len(losses): 7351
loss_mean: 0.5267899247204879
loss_standard_deviation: 0.42480818090509104


In [86]:
evaluation_data = scaled_dataset
autoencoder.evaluate(evaluation_data, n_std=0.5, tablefmt='latex_raw')

14702
Using n_std: 0.5 as prediction threshold
5698
Using n_std: 0.5 as prediction threshold
7358
Using n_std: 0.5 as prediction threshold
4312
Using n_std: 0.5 as prediction threshold
7704
Using n_std: 0.5 as prediction threshold
5687
Using n_std: 0.5 as prediction threshold
4162
Using n_std: 0.5 as prediction threshold
9381
Using n_std: 0.5 as prediction threshold
-----------
\begin{tabular}{lllllr}
\hline
 Behavior                 & Accuracy   & Precision      & Recall         & F1-Score       &   \#Samples \\
\hline
 NORMAL                   & 87.82\%    & \notCalculated & \notCalculated & \notCalculated &       14702 \\
 ROOTKIT\_BDVL            & 99.70\%    & \notCalculated & \notCalculated & \notCalculated &        5698 \\
 ROOTKIT\_BEURK           & 100.00\%   & \notCalculated & \notCalculated & \notCalculated &        7358 \\
 CNC\_BACKDOOR\_JAKORITAR & 100.00\%   & \notCalculated & \notCalculated & \notCalculated &        4312 \\
 CNC\_THETICK             & 100.00\%   & \notC

In [82]:
evaluation_data = scaled_dataset
autoencoder.evaluate(evaluation_data, n_std=0.5, tablefmt='latex_raw')

14702
Using n_std: 0.5 as prediction threshold
5698
Using n_std: 0.5 as prediction threshold
7358
Using n_std: 0.5 as prediction threshold
4312
Using n_std: 0.5 as prediction threshold
7704
Using n_std: 0.5 as prediction threshold
5687
Using n_std: 0.5 as prediction threshold
4162
Using n_std: 0.5 as prediction threshold
9381
Using n_std: 0.5 as prediction threshold
-----------
\begin{tabular}{lllllr}
\hline
 Behavior                 & Accuracy   & Precision      & Recall         & F1-Score       &   \#Samples \\
\hline
 NORMAL                   & 87.36\%    & \notCalculated & \notCalculated & \notCalculated &       14702 \\
 ROOTKIT\_BDVL            & 99.84\%    & \notCalculated & \notCalculated & \notCalculated &        5698 \\
 ROOTKIT\_BEURK           & 100.00\%   & \notCalculated & \notCalculated & \notCalculated &        7358 \\
 CNC\_BACKDOOR\_JAKORITAR & 100.00\%   & \notCalculated & \notCalculated & \notCalculated &        4312 \\
 CNC\_THETICK             & 100.00\%   & \notC

In [77]:
autoencoder.pretrain(X_normal, optimizer=torch.optim.Adam(autoencoder.parameters(), lr=1e-4,  weight_decay=0.01), loss_function=torch.nn.MSELoss(reduction="mean"), num_epochs=100, batch_size=64, verbose=False)

len(losses): 7351
loss_mean: 0.009242754975595305
loss_standard_deviation: 0.008484621480753409


In [78]:
evaluation_data = scaled_dataset
autoencoder.evaluate(evaluation_data, n_std=0.5, tablefmt='latex_raw')

14702
Using n_std: 0.5 as prediction threshold
5698
Using n_std: 0.5 as prediction threshold
7358
Using n_std: 0.5 as prediction threshold
4312
Using n_std: 0.5 as prediction threshold
7704
Using n_std: 0.5 as prediction threshold
5687
Using n_std: 0.5 as prediction threshold
4162
Using n_std: 0.5 as prediction threshold
9381
Using n_std: 0.5 as prediction threshold
-----------
\begin{tabular}{lllllr}
\hline
 Behavior                 & Accuracy   & Precision      & Recall         & F1-Score       &   \#Samples \\
\hline
 NORMAL                   & 0.00\%     & \notCalculated & \notCalculated & \notCalculated &       14702 \\
 ROOTKIT\_BDVL            & 100.00\%   & \notCalculated & \notCalculated & \notCalculated &        5698 \\
 ROOTKIT\_BEURK           & 100.00\%   & \notCalculated & \notCalculated & \notCalculated &        7358 \\
 CNC\_BACKDOOR\_JAKORITAR & 100.00\%   & \notCalculated & \notCalculated & \notCalculated &        4312 \\
 CNC\_THETICK             & 100.00\%   & \notC

In [27]:
def get_scaled_train_test_split(split=0.8, scaling_minmax=True, scale_normal_only=True, filter_outliers=True):
    """
    Method returns dictionaries mapping behaviors to scaled train and test data, as well as the scaler used
    Either decision states or raw behaviors can be utilized (decision flag) as no combinations
    with mtd need to be considered
    """
    #print(os.getcwd())
    rdf = parse_no_mtd_behavior_data(filter_outliers=filter_outliers)
    print(f"type(rdf): {type(rdf)}")
    print(f"rdf.columns: {rdf.columns}; {len(rdf.columns)}")
    # take split of all behaviors, concat, calc scaling, scale both train and test split
    train_filtered, df_test = filter_train_split_for_outliers(rdf, Behavior.NORMAL, split)
    print(f"type(train_filtered): {type(train_filtered)}")
    print(f"type(df_test): {type(df_test)}")

    # get behavior dicts for train and test
    train_bdata = {}
    test_bdata = {}
    for b in rdf["attack"].unique():
        dtrain_filtered, df_test = filter_train_split_for_outliers(rdf, b, split)
        train_bdata[b] = dtrain_filtered
        test_bdata[b] = df_test
        if b != Behavior.NORMAL and not scale_normal_only:
            train_filtered = np.vstack((train_filtered, train_bdata[b]))

    # fit scaler on either just normal data (if scale_normal_only), or all training data combined
    scaler = StandardScaler() if not scaling_minmax else MinMaxScaler()
    print(f"Using scaler {scaler}")
    scaler.fit(train_filtered[:, :-1])

    # get behavior dicts for scaled train and test data
    scaled_train = {}
    scaled_test = {}
    for b, d in train_bdata.items():
        scaled_train[b] = np.hstack((scaler.transform(d[:, :-1]), np.expand_dims(d[:, -1], axis=1)))
        scaled_test[b] = np.hstack(
            (scaler.transform(test_bdata[b][:, :-1]), np.expand_dims(test_bdata[b][:, -1], axis=1)))

    # return also scaler in case of using the agent for online scaling
    return scaled_train, scaled_test, scaler

In [28]:
def split_data(data, split=0.8):
    row = int(len(data) * split)
    X_train = data[:row, :-1].astype(np.float32)
    X_valid = data[row:, :-1].astype(np.float32)
    return X_train, X_valid

In [29]:
import numpy as np

def get_test_dataset(test_data):
    test_data_dict = {}

    for behavior, behavior_data in test_data.items():
        if behavior == Behavior.NORMAL:
            behavior_data = behavior_data[:2800]
            #continue
        else:
            behavior_data = behavior_data[:400]

        test_data_dict[behavior] = behavior_data

    return test_data_dict

In [32]:
training_data, test_data, _ = get_scaled_train_test_split(scaling_minmax=True,
                                                                         scale_normal_only=True)
normal_data = training_data[Behavior.NORMAL]
threshold = int(len(normal_data) * 0.5)

training_data[Behavior.NORMAL] = normal_data[:threshold]

ae_training_data = normal_data[threshold:]  # use remaining samples for autoencoder
ae_training_x, ae_valid_x = split_data(ae_training_data)

N_FEATURES = normal_data.shape[1] -1
flattend_test_data = np.empty([0, N_FEATURES+1])
for behavior, behavior_data in test_data.items():
    if behavior == Behavior.NORMAL:
        NR_SAMPLES = 2800
        behavior_data[:, -1] =  0
    else:
        NR_SAMPLES = 400
        behavior_data[:, -1] = 1
    #y_true = np.array([0 if behavior == Behavior.NORMAL else 1] * NR_SAMPLES)
    
    flattend_test_data = np.concatenate((flattend_test_data, behavior_data[:NR_SAMPLES]), axis=0)

ae_test_x = flattend_test_data[:,:-1]
ae_test_y = flattend_test_data[:,-1].astype(int)

evaluation_data = {}
for behavior, behavior_data in training_data.items():
    if behavior == Behavior.NORMAL:
        evaluation_data[behavior] = behavior_data[:2800]
    else:
        evaluation_data[behavior] = behavior_data[:400]

getting Behavior.NORMAL
getting Behavior.RANSOMWARE_POC
getting Behavior.ROOTKIT_BDVL
getting Behavior.ROOTKIT_BEURK
getting Behavior.CNC_THETICK
getting Behavior.CNC_BACKDOOR_JAKORITAR
getting Behavior.CNC_OPT1
getting Behavior.CNC_OPT2
type(rdf): <class 'pandas.core.frame.DataFrame'>
rdf.columns: Index(['connectivity', 'cpuUser', 'cpuSystem', 'cpuIdle', 'cpuIowait',
       'cpuSoftIrq', 'tasks', 'tasksRunning', 'tasksSleeping', 'tasksZombie',
       'ramFree', 'ramUsed', 'ramCache', 'memAvail', 'iface0RX', 'iface0TX',
       'iface1RX', 'iface1TX', 'numEncrypted', 'block:block_bio_backmerge',
       'block:block_bio_remap', 'block:block_dirty_buffer',
       'block:block_getrq', 'block:block_touch_buffer', 'block:block_unplug',
       'clk:clk_set_rate', 'cpu-migrations', 'cs', 'fib:fib_table_lookup',
       'filemap:mm_filemap_add_to_page_cache', 'gpio:gpio_value',
       'ipi:ipi_raise', 'irq:irq_handler_entry', 'irq:softirq_entry',
       'jbd2:jbd2_handle_start', 'jbd2:jbd2_start

/var/folders/8v/42fl0kws5cs5j84mbf8mbhbh0000gn/T/ipykernel_22746/2293243636.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_filtered["attack"] = b
/var/folders/8v/42fl0kws5cs5j84mbf8mbhbh0000gn/T/ipykernel_22746/2293243636.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_filtered["attack"] = b


type(train_filtered): <class 'numpy.ndarray'>
type(df_test): <class 'numpy.ndarray'>


/var/folders/8v/42fl0kws5cs5j84mbf8mbhbh0000gn/T/ipykernel_22746/2293243636.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_filtered["attack"] = b
/var/folders/8v/42fl0kws5cs5j84mbf8mbhbh0000gn/T/ipykernel_22746/2293243636.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_filtered["attack"] = b
/var/folders/8v/42fl0kws5cs5j84mbf8mbhbh0000gn/T/ipykernel_22746/2293243636.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index

Using scaler MinMaxScaler()
